In [ ]:
import tensorflow as tf
import input_data
import numpy as np
from tqdm import tqdm
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

In [ ]:
## Visualizing reconstructions
def vis(images, save_name):
  dim = images.shape[0]
  n_image_rows = int(np.ceil(np.sqrt(dim)))
  n_image_cols = int(np.ceil(dim * 1.0/n_image_rows))
  gs = gridspec.GridSpec(n_image_rows,n_image_cols,top=1., bottom=0., right=1., left=0., hspace=0., wspace=0.)
  for g,count in zip(gs,range(int(dim))):
    ax = plt.subplot(g)
    ax.imshow(images[count,:].reshape((28,28)))
    ax.set_xticks([])
    ax.set_yticks([])
  plt.savefig(save_name + '_vis.png')

In [ ]:
## Model 
X = tf.placeholder(tf.float32, shape=[None,trX.shape[1]], name='train_input')
Y = tf.placeholder(tf.float32, shape=[None,trX.shape[1]], name='train_output')

struct = [trX.shape[1],trX.shape[1]//2,100,trX.shape[1]//2,trX.shape[1]]

w = list()
b = list()
H = [X]
for i in range(len(struct) - 1):
  w.append(tf.Variable(tf.random_normal([struct[i],struct[i+1]], stddev=0.1), name='w%d' % i))
  b.append(tf.Variable(tf.zeros([struct[i+1]]), name='b%d' % i))
  H.append(tf.sigmoid(tf.add(tf.matmul(H[i],w[i]),b[i])))

loss = tf.reduce_mean(tf.squared_difference(H[-1],Y))
train_op = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
predict_op = H[-1]

In [ ]:
num_epochs = 10
batch_size = 1000
with tf.Session() as sess:
  tf.initialize_all_variables().run()
  for i in range(num_epochs):
    av_l = 0 # average loss
    count = 0
    for start, end in zip(range(0, len(trX), batch_size), range(batch_size, len(trX)+1, batch_size)):
      _, l = sess.run([train_op, loss], feed_dict={X:trX[start:end], Y:trX[start:end]})
      av_l+=l
      count+=1
    
    av_l = (1.0*av_l)/count
    print ("Epoch:%d, Train Loss:%f") %(i,av_l)
    
  predicted_images = sess.run(predict_op, feed_dict={X:teX[:100]})
  vis(predicted_images, "pred")
  vis(teX[:100], "input")
  print("Done")